In [10]:
from sqlalchemy import create_engine, MetaData

In [11]:
engine = create_engine("sqlite:///alchemy_core.db",echo=True)

In [12]:
conn = engine.connect()

2018-07-10 16:38:20,516 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:38:20,516 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:38:20,518 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 16:38:20,519 INFO sqlalchemy.engine.base.Engine ()


In [13]:
metadata = MetaData(bind = engine, reflect= True)

2018-07-10 16:38:21,487 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 16:38:21,488 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:38:21,490 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2018-07-10 16:38:21,491 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:38:21,493 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-10 16:38:21,494 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:38:21,496 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Album")
2018-07-10 16:38:21,497 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 16:38:21,499 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2018-07-10 16:38:21,500 INFO sqlalchemy.engine.base.Engine ()
2018-07

C:\Users\hangs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  """Entry point for launching an IPython kernel.


In [14]:
metadata.reflect(bind = engine)

2018-07-10 16:38:26,444 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-07-10 16:38:26,446 INFO sqlalchemy.engine.base.Engine ()


In [15]:
for row in metadata.tables:
    print(row)

Album
Artist
Genre
Track


In [17]:
album = metadata.tables["Album"]

In [18]:
result = conn.execute(album.select())

2018-07-10 16:38:41,584 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2018-07-10 16:38:41,585 INFO sqlalchemy.engine.base.Engine ()


In [19]:
for row in result:
    print(row)

(1, 'IV', 1)
(2, 'Who Made Who', 2)


# 이제 진짜 ORM Practical Using

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite://", echo=True)

In [3]:
from sqlalchemy.ext.declarative import declarative_base

In [4]:
Base = declarative_base()

In [5]:
from sqlalchemy import Column, Integer, String, ForeignKey

In [7]:

class Artist(Base):
    __tablename__="artist"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    def __repr__(self):
        return "<T'artist(name='%s')> " %(self.name)

class Album(Base):
    __tablename__="album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))
    
class Genre(Base) :
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)

class Track(Base) :
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))
    
    
    
    

In [8]:
Base.metadata.create_all(engine)

2018-07-10 17:23:14,437 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:23:14,440 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,442 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:23:14,443 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,445 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-10 17:23:14,446 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,448 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-10 17:23:14,449 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,450 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-10 17:23:14,451 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,452 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-10 17:23:14,453 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:23:14,455 INFO sqlalchemy.engin

In [9]:
artist1 = Artist(name="Led")
artist2 = Artist(name="Led2")

In [10]:
artist1

<T'artist(name='Led')> 

In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
Session = sessionmaker(bind=engine)

In [13]:
session = Session()

In [14]:
session.add_all([artist1, artist2])

In [15]:
session.commit()

2018-07-10 17:23:20,576 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:23:20,579 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:23:20,580 INFO sqlalchemy.engine.base.Engine ('Led',)
2018-07-10 17:23:20,581 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:23:20,582 INFO sqlalchemy.engine.base.Engine ('Led2',)
2018-07-10 17:23:20,584 INFO sqlalchemy.engine.base.Engine COMMIT


In [44]:
session.dirty

IdentitySet([])

In [26]:
session.new

IdentitySet([<__main__.Album object at 0x000001EAF2D8C518>, <__main__.Album object at 0x000001EAF2D8C9B0>])

In [21]:
artist1.name = 'xyz'

In [22]:
session.dirty

IdentitySet([<T'artist(name='xyz')> ])

In [24]:
session.add_all([
    Album(title="1", artist_id=1),
    Album(title="2", artist_id=2),
])

In [30]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal")
])

In [31]:
session.add_all([
    Track(title="1-1",album_id=1,genre_id=1),
    Track(title="1-2",album_id=1,genre_id=2),
    Track(title="1-3",album_id=1,genre_id=3),
    Track(title="1-4",album_id=1,genre_id=4),
])

In [32]:
session.new

IdentitySet([<__main__.Album object at 0x000001EAF2D8C518>, <__main__.Album object at 0x000001EAF2D8C9B0>, <__main__.Genre object at 0x000001EAF2DC9358>, <__main__.Genre object at 0x000001EAF2DC93C8>, <__main__.Track object at 0x000001EAF2DC9B00>, <__main__.Track object at 0x000001EAF2DC9B70>, <__main__.Track object at 0x000001EAF2DC9BE0>, <__main__.Track object at 0x000001EAF2DC9C50>])

In [33]:
session.commit()

2018-07-10 17:29:25,344 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:29:25,346 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-10 17:29:25,347 INFO sqlalchemy.engine.base.Engine ('Rock',)
2018-07-10 17:29:25,349 INFO sqlalchemy.engine.base.Engine INSERT INTO genre (name) VALUES (?)
2018-07-10 17:29:25,350 INFO sqlalchemy.engine.base.Engine ('Metal',)
2018-07-10 17:29:25,352 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-10 17:29:25,353 INFO sqlalchemy.engine.base.Engine ('1', 1)
2018-07-10 17:29:25,354 INFO sqlalchemy.engine.base.Engine INSERT INTO album (title, artist_id) VALUES (?, ?)
2018-07-10 17:29:25,355 INFO sqlalchemy.engine.base.Engine ('2', 2)
2018-07-10 17:29:25,358 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:29:25,359 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-10 17:29:25,361 INFO sqlalchemy.engine.base.

In [34]:
temp = Track(title="3-1",album_id=3,genre_id=3)

In [41]:
print(temp.id)

2018-07-10 17:31:32,016 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:31:32,019 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-07-10 17:31:32,020 INFO sqlalchemy.engine.base.Engine (5,)
5


In [37]:
session.add(temp)

In [40]:
session.new

IdentitySet([])

In [39]:
session.commit()

2018-07-10 17:31:16,537 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:31:16,539 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, album_id, genre_id) VALUES (?, ?, ?)
2018-07-10 17:31:16,540 INFO sqlalchemy.engine.base.Engine ('3-1', 3, 3)
2018-07-10 17:31:16,542 INFO sqlalchemy.engine.base.Engine COMMIT


In [45]:
for row in session.query(Artist).filter(Artist.id==1):
    print(row.id, row.name)

2018-07-10 17:34:56,148 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:34:56,149 INFO sqlalchemy.engine.base.Engine (1,)
1 xyz
